In [2]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('corrupt_records').getOrCreate()

In [6]:
data = """ {"a":1,"b":2,"c":3} | {"a":4,"b":5,"c":6} | {"a":1,"b,"c":3}  """.split('|')

In [7]:
data

[' {"a":1,"b":2,"c":3} ', ' {"a":4,"b":5,"c":6} ', ' {"a":1,"b,"c":3}  ']

In [16]:
corrupt_values = spark.read.option('mode','PERMISSIVE').json(spark.sparkContext.parallelize(data))

In [14]:
corrupt_values.show()

+-------------------+----+----+----+
|    _corrupt_record|   a|   b|   c|
+-------------------+----+----+----+
|               null|   1|   2|   3|
|               null|   4|   5|   6|
| {"a":1,"b,"c":3}  |null|null|null|
+-------------------+----+----+----+



In [17]:
spark.read.option('mode','DROPMALFORMED').json(spark.sparkContext.parallelize(data)).show()

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1|  2|  3|
|  4|  5|  6|
+---+---+---+



In [18]:

try:
    spark.read.option('mode','FAILFAST').json(spark.sparkContext.parallelize(data)).show()
except Exception as e:
    print(e)

An error occurred while calling o310.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 7.0 failed 1 times, most recent failure: Lost task 1.0 in stage 7.0 (TID 11, localhost, executor driver): org.apache.spark.SparkException: Malformed records are detected in schema inference. Parse Mode: FAILFAST.
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:66)
	at org.apache.spark.sql.catalyst.json.JsonInferSchema$$anonfun$1$$anonfun$apply$1.apply(JsonInferSchema.scala:53)
	at scala.collection.Iterator$$anon$12.nextCur(Iterator.scala:435)
	at scala.collection.Iterator$$anon$12.hasNext(Iterator.scala:441)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1334)
	at scala.collection.TraversableOnce$class.reduceLeft(TraversableOnce.scala:185)
	at scala.collection.AbstractIterator.reduceLeft(Iterator.scala:1334)
	at scala.col

In [23]:
spark.read.option("badRecordsPath", '/home/amogh/Desktop/data/').json(spark.sparkContext.parallelize(data))

DataFrame[_corrupt_record: string, a: bigint, b: bigint, c: bigint]